## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

====================================================================================================

## 1

import libs \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file\
memory allocations to manage ghosts


====================================================================================================

In [2]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import csv


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T()

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc()

agent_cache={} # dict with uid as keys and agents' tuples as values

===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [3]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, labor:int, capital:float,\
                 duration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 averageServiceLifeOfAssets: float, markup: float, dimensionalClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        self.labor=labor
        self.capital=capital
        self.duration=duration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.dimensionalClass=dimensionalClass
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP=aProductiveProcess
        self.availableLabor=self.labor
        self.availableCapital=self.capital
        self.averageServiceLifeOfAssets=averageServiceLifeOfAssets
        self.markup=markup

        
        self.profits=0
        self.revenues=0
        self.totalCosts=0
        self.totalCostOfLabor=0
        self.totalCostOfCapital=0
        self.addedValue=0
        self.finalInventories=0
        self.initialInventories=0
        
        self.myBalancesheet=np.zeros((params['howManyCycles'], 13))

        
    def produce(self, productionOrder:float)->tuple:
        
        self.costOfProductionOrder=0
        self.output=0
        self.costOfUnusedFactors=0
        self.totalLostProduction=0
        self.totalCostOfLostProduction=0
        
        
        lastOrderProductionInEachPeriod=productionOrder/self.duration
        requiredLabor=np.ceil(lastOrderProductionInEachPeriod/self.laborProductivity)
        requiredCapital=requiredLabor*self.recipe
        
        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapital <= self.capital: 
            aProductiveProcess = ProductiveProcess(lastOrderProductionInEachPeriod, \
                                                   requiredLabor, requiredCapital, self.duration)
            self.appRepository.append(aProductiveProcess)
        
        for aProductiveProcess in self.appRepository:  
            
            if aProductiveProcess.hasResources or \
            (self.availableLabor >= aProductiveProcess.requiredLabor and\
                self.availableCapital >= aProductiveProcess.requiredCapital):
                
                if not aProductiveProcess.hasResources:
                    self.availableLabor -= aProductiveProcess.requiredLabor
                    self.availableCapital -= aProductiveProcess.requiredCapital
                    aProductiveProcess.hasResources = True 
                #production
                (outputOfThePeriod, requiredLabor, requiredCapital,\
                lostProduction, costOfLostProduction)=aProductiveProcess.step(productionOrder)               
                     
                self.output += outputOfThePeriod
                
                cost = requiredLabor*params['wage'] + requiredCapital*params['costOfCapital']
                self.costOfProductionOrder += cost
                
                self.totalLostProduction += lostProduction
                self.totalCostOfLostProduction += costOfLostProduction
              
        
                if aProductiveProcess.failure:
                    self.inProgressInventories -= cost*(aProductiveProcess.productionClock-1)
                
                else:
                    if aProductiveProcess.productionClock < aProductiveProcess.duration:
                        self.inProgressInventories += cost
                    else:
                        self.inventories+=cost*self.duration
                        self.inProgressInventories -= cost*(self.duration-1)

        self.costOfUnusedFactors =  self.availableLabor*params['wage'] + \
                                    self.availableCapital*params['costOfCapital']
        
        self.totalCostOfLabor= self.labor*params['wage']
        self.totalCostOfCapital=self.capital*params['costOfCapital']
        
        print("LABOR",self.uid, self.capital / requiredCapital, self.labor, flush=True)
        print("CAPITAL",self.uid, self.labor / requiredLabor, self.capital, flush=True)
        
        if  self.capital / requiredCapital > params['unusedCriticalRatio']:
            self.capital *= (1 - 1/self.averageServiceLifeOfAssets) #no substitution of the machinery
            #self.totalCostOfCapital += #subtstitution
       
        if self.labor / requiredLabor > params['unusedCriticalRatio']:
            self.labor *= (1/(1+params['changeInStockOfWorkers']))
        else:
            self.labor *= (1+ params['changeInStockOfWorkers'])
        self.labor=int(self.labor)
            
        # remove concluded aPPs from the list
        for i in range(len(self.appRepository)-1,-1,-1):
            if self.appRepository[i].productionClock == self.appRepository[i].duration: 
                self.availableLabor+=self.appRepository[i].requiredLabor
                self.availableCapital+=self.appRepository[i].requiredCapital
                #self.appRepository.remove(aProductiveProcess)
                del self.appRepository[i]

        
        #print(self.uid,"\n", productionOrder,  self.labor, self.capital, self.recipe,\
              #self.laborProductivity,self.duration,"\n",\
              #output, self.costOfProductionOrder, self.costOfUnusedFactors, self.inventories,\
              #self.inProgressInventories, self.totalLostProduction,self.totalCostOfLostProduction, flush=True)
        
        return(self.output, self.costOfProductionOrder, self.costOfUnusedFactors,self.inventories,\
               self.inProgressInventories, self.totalLostProduction, self.totalCostOfLostProduction)
    
    def makeBalancesheet(self):
        if t()==0: self.initialInventories=0 
        else: self.initialInventories=self.finalInventories
        
        self.totalCosts=self.costOfProductionOrder + self.costOfUnusedFactors
        if params['usingMarkup']: self.finalInventories /= (1+self.markup) 
        self.finalInventories=self.inventories+self.inProgressInventories
        if params['usingMarkup']: self.finalInventories *= (1+self.markup) #to open the possibility of having a floating markup
            
        self.profits=self.finalInventories-self.totalCosts-self.initialInventories
        self.addedValue=self.profits+self.totalCosts
        
        self.myBalancesheet[t(), 0]=self.dimensionalClass
        
        self.myBalancesheet[t(), 1]=self.initialInventories
        self.myBalancesheet[t(), 2]=self.totalCosts
        self.myBalancesheet[t(), 3]=self.finalInventories
        self.myBalancesheet[t(), 4]=self.profits
        
        self.myBalancesheet[t(), 5]=self.addedValue
        self.myBalancesheet[t(), 6]=self.output
        self.myBalancesheet[t(), 7]=self.costOfProductionOrder
        self.myBalancesheet[t(), 8]=self.costOfUnusedFactors
        self.myBalancesheet[t(), 9]=self.totalLostProduction
        self.myBalancesheet[t(), 10]=self.totalCostOfLostProduction
        self.myBalancesheet[t(), 11]=self.totalCostOfLabor
        self.myBalancesheet[t(), 12]=self.totalCostOfCapital
        
        
    
###########################################################################################################################


class ProductiveProcess():
    def __init__(self, targetProductionOfThePeriod:float, requiredLabor:int, requiredCapital:float, duration:int):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredLabor = requiredLabor
        self.requiredCapital = requiredCapital
        self.duration = duration
        self.productionClock=0
        self.hasResources= False
        
    def step(self, productionOrder)->tuple:
        
        lostProduction=0
        self.costOfLostProduction=0
        self.productionClock += 1
        self.failure=False
        
        # production failure
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.failure=True
            #print("failure",flush=True)
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.targetProductionOfThePeriod=0
            self.costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       params['costOfCapital']* self.requiredCapital)*self.productionClock
            self.duration = self.productionClock   

        return(self.targetProductionOfThePeriod, self.requiredLabor, self.requiredCapital, \
               lostProduction, self.costOfLostProduction)

===================================================================================================

## 3

the model

===================================================================================================

In [4]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInventories=[]
        self.totalInProgressInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initGhosts) 
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        runner.schedule_repeating_event(0.2,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L-min, L-max, K-min, K-max, t-min, t-max, recipe, L-prod
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=0
            k=0
            #for each record in firm-features.csv
            for row in firmReader:
                print(row)
                for i in range(int(row[0] * params['Firm.count'])// rankNum):
                    labor= rng.integers(row[1], row[2]+1)
                    capital= row[3] + rng.random()*(row[4] -row[3])
                    duration= rng.integers(row[5], row[6]+1)
                    recipe= row[7] #K/L 
                    laborProductivity= row[8]
                    maxOrderProduction= row[9]
                    averageServiceLifeOfAssets=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                    markup=row[11]
                    dimensionalClass=self.rowNumber
                    aFirm =Firm(k, rank, labor, capital, duration, recipe, laborProductivity,\
                                maxOrderProduction, averageServiceLifeOfAssets, markup, dimensionalClass)
                    context.add(aFirm)
                    k += 1
                self.rowNumber += 1

        
    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
  
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of dimClass
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInventories.append([0]*(self.rowNumber))
        self.totalInProgressInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            #print(aFirm.dimensionalClass)
            tupleOfProductionResults = aFirm.produce(rng.random()*aFirm.maxOrderProduction)

            self.totalProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.dimensionalClass] += tupleOfProductionResults[2]
            self.totalInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[3]
            self.totalInProgressInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[4] 
            self.totalLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[6]
            
    def enterprisesMakingBalancesheet(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.makeBalancesheet()
    
    
    #finish
    def finish(self):
        
        # infos for data_analysis.ipynb
        with open('plotInfo.csv', 'w', newline='')\
          as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum,\
                             context.size(agent_type_ids=[0])[0]))
        
        """
        print("\n total production", self.totalProduction, flush=True)
        print("\n total cost of production", self.totalCostOfProduction, flush=True)
        print("\n total cost of unused factors", self.totalCostOfUnusedFactors, flush=True)
        print("\n total inventories", self.totalInventories, flush=True)
        print("\n total in progress inventories", self.totalInProgressInventories, flush=True)
        print("\n total lost production", self.totalLostProduction, flush=True)
        print("\n total cost of lost production", self.totalCostOfLostProduction, flush=True)     
        """
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",
               "_total_inventories_","_total_in_progress_inventories_",
               "_total_lost_production_","_total_cost_of_lost_production_"]
        contents=[self.totalProduction,self.totalCostOfProduction,
                  self.totalCostOfUnusedFactors,
                  self.totalInventories,self.totalInProgressInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction]
        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])
                    
        fileList=[] 
        
        for aFirm in context.agents(agent_type=0):        
            #save balancesheets
            with open(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv', 'w', newline='') as file:
                np.savetxt(file, aFirm.myBalancesheet, delimiter=",")
                #create the list of the files containing the balancesheets
                fileList.append(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv')
        
        #save list of balancesheets file names
        with open(params["log_file_root"]+'_balancesheetList_'+str(rank)+'.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fileList)
         
        print("THE END!", flush=True)
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

==========================================================================================================

In [5]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

[0.1, 40.0, 80.0, 200.0, 400.0, 5.0, 10.0, 5.0, 0.8, 100.0, 12.0, 0.3]
[0.3, 20.0, 50.0, 60.0, 200.0, 2.0, 4.0, 2.0, 0.7, 50.0, 12.0, 0.2]
[0.6, 2.0, 18.0, 1.0, 8.0, 1.0, 1.0, 0.5, 0.6, 10.0, 12.0, 0.1]
rank 0 tick 0
LABOR (0, 0, 0) 3.2880350771619438 80
CAPITAL (0, 0, 0) 3.6363636363636362 361.68385848781384
LABOR (1, 0, 0) 3.863133959871655 49
CAPITAL (1, 0, 0) 4.9 77.2626791974331
LABOR (2, 0, 0) 3.8004693192272314 47
CAPITAL (2, 0, 0) 1.9583333333333333 182.42252732290711
LABOR (3, 0, 0) 6.0876195097162 42
CAPITAL (3, 0, 0) 2.8 182.628585291486
LABOR (4, 0, 0) 1.1351944109568606 5
CAPITAL (4, 0, 0) 1.0 2.8379860273921516
LABOR (5, 0, 0) 0.5482470526428279 10
CAPITAL (5, 0, 0) 0.6666666666666666 4.111852894821209
LABOR (6, 0, 0) 1.4498073062043835 7
CAPITAL (6, 0, 0) 0.7777777777777778 6.524132877919725
LABOR (7, 0, 0) 0.8819354107656837 9
CAPITAL (7, 0, 0) 1.5 2.6458062322970513
LABOR (8, 0, 0) 0.8621309211849267 11
CAPITAL (8, 0, 0) 2.2 2.155327302962317
LABOR (9, 0, 0) 0.84903887

CAPITAL (1, 0, 0) 4.5 35.307967313656114
LABOR (2, 0, 0) 10.420568990904428 32
CAPITAL (2, 0, 0) 8.0 83.36455192723542
LABOR (3, 0, 0) 6.954893128860584 22
CAPITAL (3, 0, 0) 3.6666666666666665 83.45871754632701
LABOR (4, 0, 0) 0.39744943207691 3
CAPITAL (4, 0, 0) 0.25 2.3846965924614603
LABOR (5, 0, 0) 1.1612970338170778 8
CAPITAL (5, 0, 0) 1.6 2.903242584542695
LABOR (6, 0, 0) 0.8445200971915584 4
CAPITAL (6, 0, 0) 0.4 4.222600485957792
LABOR (7, 0, 0) 0.6063305949014075 7
CAPITAL (7, 0, 0) 0.875 2.42532237960563
LABOR (8, 0, 0) 0.30395641452032673 7
CAPITAL (8, 0, 0) 0.5384615384615384 1.9757166943821236
LABOR (9, 0, 0) 8.56114195816924 11
CAPITAL (9, 0, 0) 11.0 4.28057097908462
rank 0 tick 10
LABOR (0, 0, 0) 1.377370677324055 28
CAPITAL (0, 0, 0) 1.2727272727272727 151.51077450564605
LABOR (1, 0, 0) 4.045704588023097 16
CAPITAL (1, 0, 0) 4.0 32.36563670418477
LABOR (2, 0, 0) 1.819464426983313 29
CAPITAL (2, 0, 0) 1.380952380952381 76.41750593329914
LABOR (3, 0, 0) 1.1250562414333296

LABOR (6, 0, 0) 0.4435196690979972 4
CAPITAL (6, 0, 0) 0.25 3.548157352783978
LABOR (7, 0, 0) 0.44464243626103217 4
CAPITAL (7, 0, 0) 0.4 2.223212181305161
LABOR (8, 0, 0) 0.3018456060861578 5
CAPITAL (8, 0, 0) 0.45454545454545453 1.6601508334738677
LABOR (9, 0, 0) 0.6594259228196329 7
CAPITAL (9, 0, 0) 0.7 3.2971296140981643
rank 0 tick 19
LABOR (0, 0, 0) 2.3079437271571255 11
CAPITAL (0, 0, 0) 1.8333333333333333 69.23831181471377
LABOR (1, 0, 0) 1.4962815292606084 10
CAPITAL (1, 0, 0) 1.4285714285714286 20.947941409648518
LABOR (2, 0, 0) 2.116468782671269 11
CAPITAL (2, 0, 0) 1.2222222222222223 38.09643808808284
LABOR (3, 0, 0) 6.356578410548811 12
CAPITAL (3, 0, 0) 4.0 38.13947046329287
LABOR (4, 0, 0) 2.003807553387755 1
CAPITAL (4, 0, 0) 0.5 2.003807553387755
LABOR (5, 0, 0) 0.6389245833376862 4
CAPITAL (5, 0, 0) 0.5714285714285714 2.236236041681902
LABOR (6, 0, 0) 0.4730876470378637 4
CAPITAL (6, 0, 0) 0.26666666666666666 3.548157352783978
LABOR (7, 0, 0) 0.2964282908406881 4
CAP

rank 0 tick 28
LABOR (0, 0, 0) 1.5820471641681455 4
CAPITAL (0, 0, 0) 1.0 31.640943283362912
LABOR (1, 0, 0) 0.2824602301925682 4
CAPITAL (1, 0, 0) 0.18181818181818182 12.428250128473001
LABOR (2, 0, 0) 2.4453298174786964 15
CAPITAL (2, 0, 0) 2.5 29.343957809744357
LABOR (3, 0, 0) 0.5649443018456507 15
CAPITAL (3, 0, 0) 0.5769230769230769 29.377103695973837
LABOR (4, 0, 0) 1.2245490604036278 1
CAPITAL (4, 0, 0) 0.3333333333333333 1.8368235906054418
LABOR (5, 0, 0) 0.5368741290545836 2
CAPITAL (5, 0, 0) 0.2857142857142857 1.8790594516910426
LABOR (6, 0, 0) 0.8518393644580581 3
CAPITAL (6, 0, 0) 0.42857142857142855 2.981437775603203
LABOR (7, 0, 0) 0.2964282908406881 4
CAPITAL (7, 0, 0) 0.26666666666666666 2.223212181305161
LABOR (8, 0, 0) 1.6601508334738677 4
CAPITAL (8, 0, 0) 2.0 1.6601508334738677
LABOR (9, 0, 0) 0.6926261862949269 5
CAPITAL (9, 0, 0) 0.625 2.7705047451797076
rank 0 tick 29
LABOR (0, 0, 0) 1.4502099004874667 4
CAPITAL (0, 0, 0) 1.0 29.004198009749334
LABOR (1, 0, 0) 0

LABOR (4, 0, 0) 0.3144048609948358 1
CAPITAL (4, 0, 0) 0.1111111111111111 1.4148218744767609
LABOR (5, 0, 0) 0.25054126022547235 2
CAPITAL (5, 0, 0) 0.13333333333333333 1.8790594516910426
LABOR (6, 0, 0) 2.2964662496110315 1
CAPITAL (6, 0, 0) 0.5 2.2964662496110315
LABOR (7, 0, 0) 0.5708131584330959 2
CAPITAL (7, 0, 0) 0.3333333333333333 1.7124394752992875
LABOR (8, 0, 0) 0.25363415511406306 2
CAPITAL (8, 0, 0) 0.18181818181818182 1.394987853127347
LABOR (9, 0, 0) 0.4232715582913442 3
CAPITAL (9, 0, 0) 0.2727272727272727 2.327993570602393
rank 0 tick 38
LABOR (0, 0, 0) 0.4550866588013759 3
CAPITAL (0, 0, 0) 0.3333333333333333 20.478899646061915
LABOR (1, 0, 0) 0.3681891230635141 1
CAPITAL (1, 0, 0) 0.07692307692307693 9.572917199651366
LABOR (2, 0, 0) 1.1207761663444025 19
CAPITAL (2, 0, 0) 1.7272727272727273 24.657075659576854
LABOR (3, 0, 0) 0.33952999322407734 9
CAPITAL (3, 0, 0) 0.32142857142857145 19.01367962054833
LABOR (4, 0, 0) 0.20211741063953728 1
CAPITAL (4, 0, 0) 0.07142857

...

CAPITAL (8, 0, 0) 0.2 1.17217729325284
LABOR (9, 0, 0) 0.7172591302018021 2
CAPITAL (9, 0, 0) 0.4 1.7931478255045052
rank 0 tick 56
LABOR (0, 0, 0) 0.11045428464780511 1
CAPITAL (0, 0, 0) 0.041666666666666664 13.254514157736613
LABOR (1, 0, 0) 0.22530394920282862 1
CAPITAL (1, 0, 0) 0.06666666666666667 6.759118476084859
LABOR (2, 0, 0) 1.044917894798681 8
CAPITAL (2, 0, 0) 1.1428571428571428 14.628850527181536
LABOR (3, 0, 0) 0.9478894392642341 3
CAPITAL (3, 0, 0) 0.6 9.47889439264234
LABOR (4, 0, 0) 0.1486054225795903 1
CAPITAL (4, 0, 0) 0.0625 1.1888433806367225
LABOR (5, 0, 0) 0.39473297509481964 1
CAPITAL (5, 0, 0) 0.125 1.5789319003792786
LABOR (6, 0, 0) 0.5404861490572773 1
CAPITAL (6, 0, 0) 0.16666666666666666 1.6214584471718319
LABOR (7, 0, 0) 0.1798656046104286 1
CAPITAL (7, 0, 0) 0.0625 1.4389248368834289
LABOR (8, 0, 0) 2.34435458650568 1
CAPITAL (8, 0, 0) 1.0 1.17217729325284
LABOR (9, 0, 0) 0.5977159418348351 2
CAPITAL (9, 0, 0) 0.3333333333333333 1.7931478255045052
rank 0

LABOR (1, 0, 0) 1.6897796190212147 1
CAPITAL (1, 0, 0) 0.5 6.759118476084859
LABOR (2, 0, 0) 0.33524449124791017 7
CAPITAL (2, 0, 0) 0.35 13.409779649916407
LABOR (3, 0, 0) 0.14223043421499543 3
CAPITAL (3, 0, 0) 0.10714285714285714 7.9649043160397435
LABOR (4, 0, 0) 0.47553735225468896 1
CAPITAL (4, 0, 0) 0.2 1.1888433806367225
LABOR (5, 0, 0) 0.6080898030685241 1
CAPITAL (5, 0, 0) 0.25 1.2161796061370482
LABOR (6, 0, 0) 0.37158422747687814 1
CAPITAL (6, 0, 0) 0.125 1.4863369099075125
LABOR (7, 0, 0) 2.6380288676196195 1
CAPITAL (7, 0, 0) 1.0 1.3190144338098098
LABOR (8, 0, 0) 0.16530705417668257 1
CAPITAL (8, 0, 0) 0.07692307692307693 1.0744958521484367
LABOR (9, 0, 0) 0.3984772945565567 2
CAPITAL (9, 0, 0) 0.2222222222222222 1.7931478255045052
rank 0 tick 66
LABOR (0, 0, 0) 2.650902831547323 1
CAPITAL (0, 0, 0) 1.0 13.254514157736613
LABOR (1, 0, 0) 0.5163215502564823 1
CAPITAL (1, 0, 0) 0.16666666666666666 6.195858603077787
LABOR (2, 0, 0) 0.47892070178272883 7
CAPITAL (2, 0, 0) 0.

CAPITAL (3, 0, 0) 0.3 7.9649043160397435
LABOR (4, 0, 0) 0.19814056343945374 1
CAPITAL (4, 0, 0) 0.08333333333333333 1.1888433806367225
LABOR (5, 0, 0) 0.17151250855778885 1
CAPITAL (5, 0, 0) 0.07692307692307693 1.1148313056256276
LABOR (6, 0, 0) 0.6244679378430869 1
CAPITAL (6, 0, 0) 0.25 1.2489358756861737
LABOR (7, 0, 0) 0.16932949569838507 1
CAPITAL (7, 0, 0) 0.08333333333333333 1.0159769741903104
LABOR (8, 0, 0) 0.39398181245442676 1
CAPITAL (8, 0, 0) 0.2 0.9849545311360669
LABOR (9, 0, 0) 0.4482869563761263 2
CAPITAL (9, 0, 0) 0.25 1.7931478255045052
rank 0 tick 75
LABOR (0, 0, 0) 0.24510544803810705 1
CAPITAL (0, 0, 0) 0.14285714285714285 8.578690681333747
LABOR (1, 0, 0) 0.4425613287912705 1
CAPITAL (1, 0, 0) 0.14285714285714285 6.195858603077787
LABOR (2, 0, 0) 0.20583042288817816 3
CAPITAL (2, 0, 0) 0.13043478260869565 9.468199452856195
LABOR (3, 0, 0) 0.1810205526372669 3
CAPITAL (3, 0, 0) 0.13636363636363635 7.9649043160397435
LABOR (4, 0, 0) 0.18289898163641885 1
CAPITAL (

CAPITAL (4, 0, 0) 0.2 1.0897730989169956
LABOR (5, 0, 0) 0.15926161508937536 1
CAPITAL (5, 0, 0) 0.07142857142857142 1.1148313056256276
LABOR (6, 0, 0) 0.3271022531559026 1
CAPITAL (6, 0, 0) 0.14285714285714285 1.144857886045659
LABOR (7, 0, 0) 0.5079884870951552 1
CAPITAL (7, 0, 0) 0.25 1.0159769741903104
LABOR (8, 0, 0) 0.1379392341058612 1
CAPITAL (8, 0, 0) 0.08333333333333333 0.8276354046351673
LABOR (9, 0, 0) 1.6437188400457965 1
CAPITAL (9, 0, 0) 0.5 1.6437188400457965
rank 0 tick 84
LABOR (0, 0, 0) 0.12255272401905352 1
CAPITAL (0, 0, 0) 0.07142857142857142 8.578690681333747
LABOR (1, 0, 0) 0.1408149682517679 1
CAPITAL (1, 0, 0) 0.045454545454545456 6.195858603077787
LABOR (2, 0, 0) 0.17533702690474437 3
CAPITAL (2, 0, 0) 0.1111111111111111 9.468199452856195
LABOR (3, 0, 0) 0.13385464197789013 1
CAPITAL (3, 0, 0) 0.04 6.692732098894506
LABOR (4, 0, 0) 0.31136374254771304 1
CAPITAL (4, 0, 0) 0.14285714285714285 1.0897730989169956
LABOR (5, 0, 0) 0.3185232301787507 1
CAPITAL (5, 0

LABOR (6, 0, 0) 0.1399270749611361 1
CAPITAL (6, 0, 0) 0.06666666666666667 1.0494530622085207
LABOR (7, 0, 0) 1.0159769741903104 1
CAPITAL (7, 0, 0) 0.5 1.0159769741903104
LABOR (8, 0, 0) 0.5517569364234448 1
CAPITAL (8, 0, 0) 0.3333333333333333 0.8276354046351673
LABOR (9, 0, 0) 3.0134845400839603 1
CAPITAL (9, 0, 0) 1.0 1.5067422700419801
rank 0 tick 93
LABOR (0, 0, 0) 0.5719127120889165 1
CAPITAL (0, 0, 0) 0.3333333333333333 8.578690681333747
LABOR (1, 0, 0) 0.10374758203372247 1
CAPITAL (1, 0, 0) 0.043478260869565216 4.772388773551234
LABOR (2, 0, 0) 0.2254333203060999 3
CAPITAL (2, 0, 0) 0.14285714285714285 9.468199452856195
LABOR (3, 0, 0) 0.3346366049447253 1
CAPITAL (3, 0, 0) 0.1 6.692732098894506
LABOR (4, 0, 0) 0.2854167640020703 1
CAPITAL (4, 0, 0) 0.14285714285714285 0.9989586740072459
LABOR (5, 0, 0) 0.20269660102284137 1
CAPITAL (5, 0, 0) 0.09090909090909091 1.1148313056256276
LABOR (6, 0, 0) 0.16145431726284934 1
CAPITAL (6, 0, 0) 0.07692307692307693 1.0494530622085207
L